In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 21 21:56:44 2017

@author: Mustafa
"""

# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
# load ascii text and covert to lowercase
filename = "/Users/mustafa/Downloads/text_compression_with_deep_learning-master/short_alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))


# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters:",  n_chars)
print ("Total Vocab: ", n_vocab)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))


# normalize
X = X / float(n_vocab)


# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# fit the model
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 21 21:54:08 2017

@author: Mustafa
"""

# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "/Users/mustafa/Downloads/text_compression_with_deep_learning-master/short_alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))


# normalize
X = X / float(n_vocab)


# one hot encode the output variable
y = np_utils.to_categorical(dataY)

('Total Characters: ', 5861)
('Total Vocab: ', 42)
('Total Patterns: ', 5821)


In [3]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# load the network weights
filename = "/Users/mustafa/Downloads/text_compression_with_deep_learning-master/weights-improvement-95-0.0502.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# pick a random seed
pattern = dataX[0]
y = dataY[0]

In [51]:
sum_all = 0.00
# generate characters
for i in range(len(dataX)):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    
    pattern = dataX[i]
    y = dataY[i]
    
    #print prediction
    #result = int_to_char[y]
    #print result, prediction[0,y]
    sum_all += numpy.log2(prediction[0,y])
    #sum_all += prediction[0,y]
    
    if i % 1000 == 0:
        print i

print "\nDone."

print -1 * ((sum_all / len(dataX)) / seq_length)

0
1000
2000
3000
4000
5000

Done.
0.29980121798
